In [66]:
import numpy as np
import time
import re

class Distance_to_Node :

    def __init__(self, vertex_number, distance_to_it) :
        self.vertex_number = vertex_number
        self.distance_to_it = distance_to_it

class Graph :
    
    def __init__(self, node_count) :
        self.adjancy_list = {}
        self.parent = []
        self.fringes = []
        self.node_count = node_count
        self.square = {}
        self.distance_to_destination_by_square = {}
        self.fringe_count = 0

    def Create_Adjancy_List (self, source, node_distance) :
        if source not in self.adjancy_list :
            self.adjancy_list[source] = []
        self.adjancy_list[source].append(node_distance)
    
    def set_square_ids (self, source, square_number) :
        self.square[source] = square_number

    def Find_Shortest_Path (self, source, destination, method) :  
        
        # if we use A* then we define heuristic function. 
        # Note : In normal, we had to multiply heuristic values by 100 to normalize edge distances. 
        #         But, I multiplied by 500 (heuristic function * 500) to get better time and space complexity and 
        #          admissible results. 
        if method == 2 :
            for it in range( len(self.square) ) :
                self.distance_to_destination_by_square[it] = 500 * np.sqrt( ( ( (self.square[destination] // 10) - (self.square[it] // 10) ) ** 2) + ( ( (self.square[destination] % 10) - (self.square[it] % 10) ) ** 2) )
        
        # initialize the distances to nodes with large numbers (in normal, infinity), because we will use dijkstra.
        distance_to_nodes = [999999999999] * self.node_count
        
        # we have to keep the path to print
        parent = [-1] * self.node_count
        
        # for each searching , we again initialize fringes to empty and count to 0
        self.fringes = []
        self.fringe_count = 0
        # the first source node distance is 0
        distance_to_nodes[source] = 0
        
        # we use dictionary to keep and find nodes with lowest total cost
        dictionary_for_distances_to_nodes = {source : 0}
        
        while dictionary_for_distances_to_nodes :
            
            # if it is A* , we add cost + heuristic function, otherwise just cost
            if method == 2 :
                source_node = min(dictionary_for_distances_to_nodes, key = lambda k : (dictionary_for_distances_to_nodes[k] + self.distance_to_destination_by_square[k] ) )
            else :
                source_node = min(dictionary_for_distances_to_nodes, key = lambda k : dictionary_for_distances_to_nodes[k])
            
            # add visited nodes to fringes
            self.fringes.append(source_node)
            self.fringe_count += 1
            # after each minimum source node found, we delete it from dictionary
            del dictionary_for_distances_to_nodes[source_node]            
            
            # for each iteration we will look its adjancy list and use dijkstra to assign shortest path to nodes.
            for adjancy_nodes in self.adjancy_list[source_node] :

                adjancy_node_number = adjancy_nodes.vertex_number
                length_to_adjancy_node = adjancy_nodes.distance_to_it

                if distance_to_nodes[adjancy_node_number] > distance_to_nodes[source_node] + length_to_adjancy_node :
                    distance_to_nodes[adjancy_node_number] = distance_to_nodes[source_node] + length_to_adjancy_node
                    dictionary_for_distances_to_nodes[adjancy_node_number] = distance_to_nodes[adjancy_node_number]
                    parent[adjancy_node_number] = source_node
            
            # when source node found destination node it stops iterating                           
            if  source_node == destination :
                break
        
        # when destination not found, function returns 0
        if source_node != destination :
            print("Path not found")
            return 0
        
            
        print("\n\nThe results using ","uninformed search (UCS) :" if method == 1 else " informed search (A*) :")
        
        # to print path
        path = source_node
        print("\nPath : ", end = "")
        while ( path != source ) :
            print(path, "<-", end = " ")
            path = parent[path] 
        print(path)
        
        # to print Fringes and their total count
        print("\nFringes count: ", len(self.fringes))
        print("Fringes: ", end = "")
        for coun in self.fringes :
            if coun == self.fringes[-1] :
                print(coun)
            else :
                print(coun, end = ",")     
        
        # to print total cost of finding the shortest path 
        print("\nThe cost from  Source Node (", str(source) ,")  to ->  Destination Node(", str(destination) ,")  is : ", str(distance_to_nodes[destination]))

def main() :
    
    # input :
    inp = input("Enter source and destination vertex : ")
    
    # create graph with 100 nodes
    graph = Graph(100)
    
    # reading v.txt file
    with open('v.txt', 'r') as reader :
        for line in reader :
            if line[0] != '#' :
                ids = int(line.split(',')[0])
                square_ids = int(line.split(',')[1])
                
                # assign vertex and square numbers 
                graph.set_square_ids(ids, square_ids)
    
    # reading e.txt file
    with open('e.txt', 'r') as reader :
        for line in reader :
            if line[0] != '#' :
                a = int(line.split(',')[0])
                b = int(line.split(',')[1])
                dist = int(line.split(',')[2])
                
                # create Adjancy list with edge distances. Graph is undirected, therefore I assigned b as a's adjacent
                #  node, and a as b's. Distance_to_Node keeps node number and distance to it.
                graph.Create_Adjancy_List(a, Distance_to_Node(b, dist))
                graph.Create_Adjancy_List(b, Distance_to_Node(a, dist))

    
    # split input
    source = int(re.split(',|;| |:', inp)[0])
    destination = int(re.split(',|;| |:', inp)[1])
    
    # to estimate UCS time
    start_time1 = time.time()
    # takes input and find the shortest path using "UNINFORMED SEARCH" on the base of source, destination nodes and searching preference
    graph.Find_Shortest_Path(source, destination, 1)

    fringe_count1 = graph.fringe_count
    
    end_time1 = time.time() 
    
    time1 = end_time1 - start_time1
    
    print("Time required to implement : ", time1, "\n\n")
        
        
        
    # to estimate A* time  
    start_time2 = time.time()
    # takes input, split and find the shortest path using "INFORMED SEARCH" on the base of source, destination nodes and searching preference
    graph.Find_Shortest_Path(source, destination, 2)
    
    fringe_count2 = graph.fringe_count
    
    end_time2 = time.time()
    
    time2 = end_time2 - start_time2
    
    print("Time required to implement : ", time2)
    
    
    #Performance measures
    print("\n\n\nPerformance comparison : ")
    print("\nA* searches", abs(fringe_count2 - fringe_count1), "fringes" , "more" if fringe_count2 > fringe_count1 else "less", 
          ": %.1f" %(max(fringe_count2, fringe_count1) / min(fringe_count2, fringe_count1) ) , " times " ,
          "more" if fringe_count2 > fringe_count1 else "less")
    
    print("\nA* searches %.5f"  %abs(time2 - time1), "seconds" , "more" if time2 > time1 else "less",
          ": %.2f" %(max(time2, time1) / min(time2, time1) ) if min(time2, time1) != 0 else max(time2, time1), " times " ,
          "slower" if time2 > time1 else "faster")
    
    

if __name__ == "__main__" : 
   main()


Enter source and destination vertex : 73 44


The results using  uninformed search (UCS) :

Path : 44 <- 31 <- 73

Fringes count:  77
Fringes: 73,9,0,38,31,59,36,2,29,61,24,32,40,47,8,23,19,71,13,22,57,34,68,88,63,53,76,50,67,77,91,30,62,69,12,41,72,87,3,55,39,18,98,92,52,6,49,64,82,93,15,90,28,1,21,58,48,78,81,17,96,35,75,11,4,85,89,60,14,99,94,43,37,79,33,27,44

The cost from  Source Node ( 73 )  to ->  Destination Node( 44 )  is :  7375
Time required to implement :  0.00902700424194336 




The results using   informed search (A*) :

Path : 44 <- 31 <- 73

Fringes count:  38
Fringes: 73,9,31,36,61,0,8,38,47,40,2,32,59,19,57,22,62,24,88,29,63,91,71,49,87,41,76,23,12,92,93,53,58,50,68,79,4,44

The cost from  Source Node ( 73 )  to ->  Destination Node( 44 )  is :  7375
Time required to implement :  0.003945112228393555



Performance comparison : 

A* searches 39 fringes less : 2.0  times  less

A* searches 0.00508 seconds less : 2.29  times  faster
